In [1]:
import pandas as pd

data = pd.read_csv("desert-island-discs-episodes.csv", sep="\t", on_bad_lines='warn')
df=data[['Castaway','Job','Book','Luxury','Artist 1','Song 1','Artist 2','Song 2','Artist 3','Song 3']]
# Before I set up the processing pipeline, let's do a quick check on the nulls in the dataset
def print_nulls(column_name):
    print(df[column_name].isnull().value_counts())

[print_nulls(i) for i in df.columns]

b'Skipping line 35: expected 26 fields, saw 28\nSkipping line 42: expected 26 fields, saw 28\nSkipping line 45: expected 26 fields, saw 42\nSkipping line 59: expected 26 fields, saw 42\nSkipping line 2235: expected 26 fields, saw 40\nSkipping line 3131: expected 26 fields, saw 28\nSkipping line 3161: expected 26 fields, saw 28\nSkipping line 3166: expected 26 fields, saw 30\n'


First we want to clean the data and then vectorize the fields of interest. There's lots of nulls in the jobs column but this is only a nice-to-have anyway.
We'll leave the nulls in but for data cleanliness we'll call them not-specified.

In [13]:
# We have so little data I want to retain as much as possible so I'll replace nans with 'not-specified'
df= df.fillna('not-specified')
#We can drop the classics since they are duplicates
df= df.drop(df.loc[df['Castaway'].str.contains('Classic')].index, axis=0)
#The first row has loaded the csv strangely so we can throw it out
df.drop(0,axis=0,inplace=True)


Now it's time to start classifying the podcasts. To do this, I plan to:
1. Vectorize the songs and average their values
2. Vectorize their professions, books and luxury items
3. Concatenate my two vectors
4. Query my vector space using a podcast embedded with the same model

NOTE: If this provides poor recommendations I'll have to add more context to the features - I could map synopsis of books, genres for songs and perhaps embed the wikipedia article of each castaway.

In [ ]:
# Vectorize song values


# Vectorize professions, books and luxury items